In [1]:
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="0, 1, 2, 4"

In [2]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_auc_score
import optuna
import gc
xgb.__version__

/home/tunguz/.local/lib/python3.12/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


'2.1.2'

In [3]:
!nvidia-smi

Fri Jan 10 10:45:21 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.06              Driver Version: 555.42.06      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Quadro RTX 5000                Off |   00000000:01:00.0  On |                  N/A |
| N/A   49C    P8              7W /  110W |     443MiB /  16384MiB |     18%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
!nvidia-smi -L

GPU 0: Quadro RTX 5000 (UUID: GPU-0dad801b-446c-e905-a909-c5a1dc218d05)


In [5]:
train_x = pd.read_csv('../input/higgs_small_roc/xgtrain_ss.csv')
test_x = pd.read_csv('../input/higgs_small_roc/xgval_ss.csv')

In [6]:
train_y = train_x['target']
train_x = train_x[train_x.columns.difference(['target'])]

test_y = test_x['target']
test_x = test_x[test_x.columns.difference(['target'])]

In [7]:
dtrain = xgb.DMatrix(train_x, train_y)

In [8]:
dtest = xgb.DMatrix(test_x, test_y)


In [9]:
num_round = 1000

In [10]:
def objective(trial):
        
    params = {
        'objective': trial.suggest_categorical('objective',['binary:logistic']),
        'tree_method': trial.suggest_categorical('tree_method',['hist']),  # 'gpu_hist','hist'
        'device': trial.suggest_categorical('device',['cuda']),

        'lambda': trial.suggest_float('lambda',1e-3,10.0, log=True),
        'alpha': trial.suggest_float('alpha',1e-3,10.0, log=True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3,1.0),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_float('learning_rate', 0.001,0.1, log=True),
        #'n_estimators': trial.suggest_categorical('n_estimators', [1000]),
        'max_depth': trial.suggest_categorical('max_depth', [3,5,7,9,11,13,15,17,20]),
        #'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'eval_metric': trial.suggest_categorical('eval_metric',['logloss']),

    }

    output = xgb.train(params, dtrain, num_round)
    
    #booster = output['booster']  # booster is the trained model
    #booster.set_param({'predictor': 'gpu_predictor'})

    predictions = output.predict(dtest)
    
    #predictions = predictions.compute()

    roc = roc_auc_score(test_y, predictions)
    
    return roc

In [11]:
study = optuna.create_study(direction='maximize')

[I 2025-01-10 10:45:30,293] A new study created in memory with name: no-name-06c4783a-c48f-4d0c-97a1-28ca8c0ab331


In [12]:
%%time
study.optimize(objective, n_trials=2)

[I 2025-01-10 10:45:33,971] Trial 0 finished with value: 0.7677356531370423 and parameters: {'objective': 'binary:logistic', 'tree_method': 'hist', 'device': 'cuda', 'lambda': 0.0032108657908986047, 'alpha': 3.7194233556865606, 'colsample_bytree': 0.6716931072096033, 'subsample': 0.947325606048736, 'learning_rate': 0.0017833707345051922, 'max_depth': 13, 'min_child_weight': 70, 'eval_metric': 'logloss'}. Best is trial 0 with value: 0.7677356531370423.
[I 2025-01-10 10:45:36,013] Trial 1 finished with value: 0.7661264247540258 and parameters: {'objective': 'binary:logistic', 'tree_method': 'hist', 'device': 'cuda', 'lambda': 0.19044688717449107, 'alpha': 0.002825209074277643, 'colsample_bytree': 0.5809601015511047, 'subsample': 0.5134729734655823, 'learning_rate': 0.00112774233856527, 'max_depth': 13, 'min_child_weight': 40, 'eval_metric': 'logloss'}. Best is trial 0 with value: 0.7677356531370423.


CPU times: user 4.89 s, sys: 196 ms, total: 5.09 s
Wall time: 4.35 s


In [13]:
%%time
study.optimize(objective, n_trials=2500)

[I 2025-01-10 10:45:39,137] Trial 2 finished with value: 0.7263700362477691 and parameters: {'objective': 'binary:logistic', 'tree_method': 'hist', 'device': 'cuda', 'lambda': 2.1875021981554097, 'alpha': 0.0011193183951347916, 'colsample_bytree': 0.9775732772189178, 'subsample': 0.40489354176307635, 'learning_rate': 0.013509718476704805, 'max_depth': 7, 'min_child_weight': 285, 'eval_metric': 'logloss'}. Best is trial 0 with value: 0.7677356531370423.
[I 2025-01-10 10:45:40,523] Trial 3 finished with value: 0.7885595499778781 and parameters: {'objective': 'binary:logistic', 'tree_method': 'hist', 'device': 'cuda', 'lambda': 1.8975399330128817, 'alpha': 0.7801682322203971, 'colsample_bytree': 0.8789225111435275, 'subsample': 0.5133122252035801, 'learning_rate': 0.01406156674695115, 'max_depth': 5, 'min_child_weight': 7, 'eval_metric': 'logloss'}. Best is trial 3 with value: 0.7885595499778781.
[I 2025-01-10 10:45:41,703] Trial 4 finished with value: 0.7409820306174761 and parameters: {

CPU times: user 3h 44s, sys: 24.2 s, total: 3h 1min 8s
Wall time: 2h 46min 7s


In [14]:
study.best_trial.params

{'objective': 'binary:logistic',
 'tree_method': 'hist',
 'device': 'cuda',
 'lambda': 0.1856231807364086,
 'alpha': 0.03653152900618455,
 'colsample_bytree': 0.9332738659705733,
 'subsample': 0.7301092057776706,
 'learning_rate': 0.008401916728914681,
 'max_depth': 7,
 'min_child_weight': 1,
 'eval_metric': 'logloss'}

In [15]:
real_test_x = pd.read_csv('../input/higgs_small_roc/xgtest_ss.csv')

In [16]:
real_test_y = real_test_x['target']
real_test_x = real_test_x[real_test_x.columns.difference(['target'])]

In [17]:
drealtest = xgb.DMatrix(real_test_x, real_test_y)

In [18]:
params = study.best_trial.params

output = xgb.train(params, dtrain, num_round)
    


predictions = output.predict(drealtest)
    
roc_auc_score(real_test_y, predictions)

0.783707281341247

In [19]:
%%time
params = study.best_trial.params
params['seed'] = 2022

output = xgb.train(params, dtrain, num_round)

predictions = output.predict(drealtest)
    
roc_auc_score(real_test_y, predictions)

CPU times: user 2.94 s, sys: 9.01 ms, total: 2.95 s
Wall time: 2.58 s


0.7832485632785768

In [21]:
%%time 
scores = []

for jj in range(15):
    print(jj)
    params = study.best_trial.params
    params['seed'] = 5*jj**3 + 137

    output = xgb.dask.train(client, params, dtrain, num_round)

    booster = output['booster']  # booster is the trained model
    booster.set_param({'predictor': 'gpu_predictor'})

    predictions = xgb.dask.predict(client, booster, drealtest)

    predictions = predictions.compute()

    scores.append(roc_auc_score(real_test_y, predictions))

0


/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)


1


/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)


2


...

13


/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)


14


/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)


CPU times: user 23.5 s, sys: 4.35 s, total: 27.9 s
Wall time: 5min 9s


In [22]:
scores

[0.8067397221533028,
 0.8066990070642219,
 0.8063826106965943,
 0.8064383115265237,
 0.8064385880739696,
 0.8062720699863317,
 0.8067426754713112,
 0.8064104376311154,
 0.8061099140433141,
 0.8061604909186776,
 0.8066853310103347,
 0.8062528838550297,
 0.8066905749760565,
 0.8061511717915344,
 0.8065482678346361]

In [24]:
import numpy as np
np.mean(scores)

0.8064481371355302